In [2]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains import MapReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq


from utils import load_pdf_chunks

c:\Legal AI Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
GROQ_API_KEY = "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"

# ------------------------------------------------------
# Global constants
# ------------------------------------------------------
VECTOR_DB_PATH = "vector_db"
KNOWLEDGE_BASE_PATH = "data/IT_Act_2000.pdf"


MAX_PAGES_TO_PROCESS = 25  

In [4]:
def build_vectorstore():
    from langchain_community.document_loaders import PyMuPDFLoader
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    if os.path.exists(VECTOR_DB_PATH):
        print("📦 Using cached vector store")
        return Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embeddings)
    
    print("🔨 Building vector store (one-time setup)...")
    loader = PyMuPDFLoader(KNOWLEDGE_BASE_PATH)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
    kb_chunks = splitter.split_documents(documents)

    vector_store = Chroma.from_documents(kb_chunks, embeddings, persist_directory=VECTOR_DB_PATH)
    print(f"✅ Vector store ready with {len(kb_chunks)} chunks")
    return vector_store


In [5]:
def inspect_vectorstore_info():
    """
    Display Chroma vector DB stats: number of vectors, collections, and sample metadata.
    """
    from chromadb import PersistentClient

    # Path must match your persist_directory
    client = PersistentClient(path=VECTOR_DB_PATH)

    collections = client.list_collections()
    print(f"🧠 Found {len(collections)} collections in Chroma DB:\n")

    for c in collections:
        print(f"📂 Collection name: {c.name}")
        print(f"   → Metadata: {c.metadata}")
        print(f"   → Vector count: {c.count()}\n")

    # Example: Inspect the 'legal_cases' or default collection
    try:
        coll = client.get_collection("langchain")  # default if no name specified
        print(f"🔍 Inspecting default collection '{coll.name}':")
        print(f"   → Count: {coll.count()} vectors")

        sample = coll.get(limit=2, include=["documents", "metadatas"])
        print("🪶 Sample documents:")
        for doc, meta in zip(sample["documents"], sample["metadatas"]):
            print(f" - {meta} :: {doc[:120]}...\n")
    except Exception as e:
        print(f"⚠️ Could not inspect default collection: {e}")

In [ ]:
vector_store = build_vectorstore()
inspect_vectorstore_info()

In [29]:
import re
import numpy as np
from typing import List, Dict, Tuple
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

# Same embedding you use elsewhere (keep consistent!)
EMBED_MODEL = "all-MiniLM-L6-v2"
embedder = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)

# Regex to catch explicit IT Act mentions/sections
IT_REGEX = re.compile(
    r"(Information\s+Technology\s+Act|IT\s+Act|Sec(?:tion)?\s*\d+[A-Z]?\s*of\s+the\s+IT\s*Act|Section\s*(66[A-Z]?|67[A-Z]?|72))",
    re.IGNORECASE
)

def build_it_centroid(queries: List[str]) -> np.ndarray:
    Q = embedder.embed_documents(queries)  # list -> list[vec]
    return np.mean(np.array(Q), axis=0, keepdims=True)  # shape (1, d)

def score_chunks_against_it(
    chunks: List[str],
    it_queries: List[str],
    tau_chunk: float = 0.70,
    tau_doc: float = 0.72,
    regex_bonus: float = 0.05,
    min_pos_chunks_for_doc: int = 2,
) -> Dict:
    """
    Returns:
      {
        "chunk_scores": List[{"idx": i, "sim": float, "regex_hit": bool, "final": float}],
        "doc_score_max": float,
        "doc_score_mean_top3": float,
        "doc_label": "IT" | "NON_IT",
        "thresholds": {"chunk": tau_chunk, "doc": tau_doc}
      }
    """
    centroid = build_it_centroid(it_queries)     # (1, d)
    C = embedder.embed_documents(chunks)         # (n, d)
    sims = cosine_similarity(np.array(C), centroid).ravel()  # (n,)

    results = []
    for i, (chunk, sim) in enumerate(zip(chunks, sims)):
        hit = bool(IT_REGEX.search(chunk))
        final = sim + (regex_bonus if hit else 0.0)
        results.append({"idx": i, "sim": float(sim), "regex_hit": hit, "final": float(final)})

    # Aggregate to doc
    finals = np.array([r["final"] for r in results])
    doc_max  = float(np.max(finals)) if len(finals) else 0.0
    doc_top3 = float(np.mean(np.sort(finals)[-3:])) if len(finals) >= 3 else doc_max

    # Decision rules (modify to taste):
    #  - IT if either doc_top3 >= tau_doc OR (doc_max >= tau_doc and at least min_pos_chunks >= tau_chunk)
    pos_chunks = int(np.sum(finals >= tau_chunk))
    is_it = (doc_top3 >= tau_doc) or (doc_max >= tau_doc and pos_chunks >= min_pos_chunks_for_doc)

    return {
        "chunk_scores": results,
        "doc_score_max": doc_max,
        "doc_score_mean_top3": doc_top3,
        "doc_label": "IT" if is_it else "NON_IT",
        "thresholds": {"chunk": tau_chunk, "doc": tau_doc},
        "pos_chunks": pos_chunks
    }

C:\Users\vansh\AppData\Local\Temp\ipykernel_24616\1108465763.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)


In [30]:
IT_QUERIES = [
    # General law references
    "Information Technology Act, 2000",
    "IT Act 2000",
    "Information Technology (Amendment) Act 2008",
    "cyber law",
    "electronic evidence",
    "computer resource",
    "digital signature",
    "electronic record",
    "intermediary liability",
    "cyber crime",
    "data privacy",
    "hacking under IT Act",
    "unauthorized access to computer system",
    "cyber security offence",

    # Core sections with meaning
    "Section 43 - damage to computer or computer system",
    "Section 65 - tampering with computer source documents",
    "Section 66 - computer related offences",
    "Section 66A - sending offensive messages through communication service",
    "Section 66B - dishonestly receiving stolen computer resources",
    "Section 66C - identity theft",
    "Section 66D - cheating by personation using computer resources",
    "Section 66E - violation of privacy",
    "Section 67 - publishing or transmitting obscene material in electronic form",
    "Section 67A - publishing sexually explicit material in electronic form",
    "Section 67B - child pornography in electronic form",
    "Section 68 - failure to comply with directions of Controller",
    "Section 69 - interception, monitoring or decryption of information",
    "Section 69A - blocking of public access to information",
    "Section 70 - protected systems",
    "Section 72 - breach of confidentiality and privacy",
    "Section 72A - disclosure of information in breach of lawful contract",
    "Section 73 - publishing false digital signature certificates",
    "Section 74 - publication for fraudulent purpose",

    # Judicial / procedural parts
    "Adjudicating officer under IT Act",
    "Cyber Appellate Tribunal",
    "Jurisdiction under IT Act",
    "Powers of Controller under IT Act",
    "Appeal to Telecom Disputes Settlement and Appellate Tribunal",
    "Electronic contracts under IT Act",
    "Penalties and adjudication under IT Act",
    "Liability of intermediaries under Section 79 of IT Act",
    "Evidence Act and electronic records under Section 65B",
    "Search and seizure of computer resources under IT Act",
]


In [43]:
# pdf_path = "data/University_Of_Delhi_vs_Shashi_Kiran_And_Ors_Etc_Etc_on_10_May_2022_1.PDF"
pdf_path = "data/University_Of_Kerala_And_Ors_Etc_vs_Merlin_J_N_And_Anr_Etc_Etc_on_17_August_2022_1.pdf"
chunks = load_pdf_chunks(pdf_path, max_pages=MAX_PAGES_TO_PROCESS)

📄 PDF has 11 pages, processing first 11 pages


In [44]:
# Suppose you already split a PDF into ~35 chunks (strings)
pdf_chunks = chunks # list[str], your own chunker

res = score_chunks_against_it(
    chunks=pdf_chunks,
    it_queries=IT_QUERIES,
    tau_chunk=0.70,
    tau_doc=0.70,
    regex_bonus=0.20
)

print("Doc label:", res["doc_label"])
print("Doc max sim:", round(res["doc_score_max"], 3),
      "Top3 mean:", round(res["doc_score_mean_top3"], 3),
      "Pos chunks:", res["pos_chunks"])

# Inspect top contributing chunks
top = sorted(res["chunk_scores"], key=lambda r: r["final"], reverse=True)[:5]
for r in top:
    print(f"[#{r['idx']:02d}] final={r['final']:.3f}  sim={r['sim']:.3f}  regex={r['regex_hit']}")


Doc label: NON_IT
Doc max sim: 0.381 Top3 mean: 0.375 Pos chunks: 0
[#59] final=0.381  sim=0.381  regex=False
[#52] final=0.373  sim=0.373  regex=False
[#51] final=0.371  sim=0.371  regex=False
[#54] final=0.355  sim=0.355  regex=False
[#57] final=0.353  sim=0.353  regex=False


In [5]:
import pandas as pd
df = pd.read_csv("dataset/classification_results.csv")

In [8]:
df[df['doc_label'] == 'IT']

,pdf_name,doc_label,doc_max_sim,doc_top3_mean,positive_chunks,total_chunks,outcome_score,procedure_score,section_score
0,Ajay_Murlidhar_Batheja_vs_The_State_Of_Maharas...,IT,0.892,0.889,14,40,0.516,0.450,0.399
1,Arjun_Panditrao_Khotkar_vs_Kailash_Kushanrao_G...,IT,0.895,0.818,3,40,0.457,0.459,0.462
2,Atul_Kumar_Singh_Atul_Rai_vs_State_Of_U_P_And_...,IT,0.870,0.825,4,11,0.514,0.519,0.465
3,Awadhesh_Kumar_Paras_Nath_Pathak_vs_The_State_...,IT,0.909,0.877,10,40,0.422,0.495,0.555
4,Cyber_Crime_Police_Station_vs_Shivalinga_on_11...,IT,0.774,0.760,5,21,0.579,0.494,0.296
